# ChromaDB: Demonstração de Busca Semântica

## Busca Semântica em Documentos de Texto

Este notebook demonstra de forma didática as capacidades do **ChromaDB** para realizar busca semântica em documentos de texto em português.

### Objetivos desta demonstração:

1. **Processar documentos** - Dividir textos em chunks (parágrafos)
2. **Gerar embeddings** - Transformar texto em vetores numéricos
3. **Armazenar no ChromaDB** - Persistir embeddings e metadados
4. **Busca semântica** - Encontrar documentos semanticamente similares

### Recursos:

- [ChromaDB Documentation](https://docs.trychroma.com/)
- [Sentence Transformers](https://www.sbert.net/)


## 1. Instalação e Configuração

Instalar as dependências necessárias para o Google Colab.

In [ ]:
# Instalação de dependências
!pip install -q chromadb sentence-transformers numpy pandas matplotlib seaborn plotly scikit-learn

In [ ]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import re
from pathlib import Path
from typing import List, Dict, Tuple
import warnings

warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 2. Corpus Sintético em Português

Criamos um conjunto de documentos de texto em português.

In [ ]:
def create_sample_portuguese_texts():
    """
    Cria textos de exemplo em português..
    """
    texts = [
        {
            "id": "001",
            "title": "Tecnologia Brasileira em Ascensão",
            "content": "A tecnologia brasileira está em franco crescimento. Startups de São Paulo e outras cidades brasileiras têm desenvolvido soluções inovadoras em inteligência artificial, machine learning e cloud computing. O investimento em tecnologia no Brasil cresceu 45% no último ano.\n\nA formação de profissionais qualificados nas universidades tem impulsionado esse setor estratégico para a economia. Programas de aceleração e incubadoras apoiam empreendedores desde a concepção da ideia até a escalabilidade do negócio.",
            "category": "tecnologia"
        },
        {
            "id": "002",
            "title": "Agricultura Sustentável no Brasil",
            "content": "O agronegócio brasileiro está adotando práticas mais sustentáveis. Agricultores utilizam técnicas de agricultura de precisão, com drones e sensores IoT para monitorar plantações. O Brasil é líder mundial na produção de alimentos orgânicos certificados.\n\nA tecnologia blockchain começa a ser usada para rastreabilidade de produtos agrícolas. Sistemas de irrigação inteligente economizam água e energia, respondendo a condições climáticas em tempo real.",
            "category": "agronegocio"
        },
        {
            "id": "003",
            "title": "Educação Digital Transforma Ensino",
            "content": "A educação digital revoluciona o ensino no Brasil. Plataformas online democratizam o acesso ao conhecimento em regiões remotas. Professores utilizam ferramentas de gamificação para engajar alunos.\n\nA pandemia acelerou a adoção de metodologias ativas nas instituições de ensino. Ferramentas de videoconferência tornaram-se essenciais no ambiente educacional.""",
            "category": "educacao"
        },
        {
            "id": "004",
            "title": "Energia Renovável Avança no País",
            "content": "O Brasil amplia sua matriz de energia renovável. Parques eólicos no Nordeste geram eletricidade limpa para milhões de residências. Painéis solares se popularizam com financiamentos acessíveis.\n\nPesquisas desenvolvem tecnologias de hidrogênio verde. O país tem recursos naturais abundantes para liderar a transição energética global.",
            "category": "energia"
        },
        {
            "id": "005",
            "title": "Inteligência Artificial na Saúde",
            "content": "A inteligência artificial transforma a saúde brasileira. Algoritmos auxiliam médicos no diagnóstico precoce de doenças. Telemedicina conecta especialistas a pacientes em áreas remotas.\n\nPesquisadores desenvolvem modelos de IA adaptados para doenças tropicais. Sistemas de predição identificam surtos epidemiológicos precocemente.",
            "category": "saude"
        },
        {
            "id": "006",
            "title": "Fintechs Democratizam Serviços",
            "content": "Fintechs brasileiras lideram inovação na América Latina. Aplicativos de pagamento digital atendem milhões sem conta bancária tradicional. PIX revolucionou pagamentos instantâneos.\n\nOpen banking permite integração entre instituições. A regulação do Banco Central impulsiona competição e beneficia consumidores.",
            "category": "financas"
        },
        {
            "id": "007",
            "title": "Mobilidade Urbana Inteligente",
            "content": "Tecnologia revoluciona a mobilidade urbana. Aplicativos de transporte compartilhado reduzem uso de veículos particulares. Sistemas inteligentes otimizam fluxo de trânsito.\n\nPlataformas digitais integram diferentes modais de transporte. Análise de dados em tempo real melhora planejamento de rotas.",
            "category": "mobilidade"
        },
        {
            "id": "008",
            "title": "E-commerce em Expansão",
            "content": "O comércio eletrônico brasileiro cresce exponencialmente. Pequenos comerciantes vendem em marketplaces nacionais. Inteligência artificial personaliza recomendações de produtos.\n\nLogística avançada garante entregas rápidas. Live commerce une entretenimento e vendas online.",
            "category": "comercio"
        },
        {
            "id": "009",
            "title": "Cibersegurança em Foco",
            "content": "Cibersegurança torna-se prioridade empresarial. Ataques cibernéticos aumentam em sofisticação. Empresas investem em firewalls e autenticação multifator.\n\nLei Geral de Proteção de Dados impõe regras rigorosas. Startups desenvolvem tecnologias nacionais de segurança digital.",
            "category": "seguranca"
        },
        {
            "id": "010",
            "title": "Cloud Computing Transforma Negócios",
            "content": "Computação em nuvem revoluciona negócios. Empresas migram infraestrutura reduzindo custos. SaaS democratiza acesso a softwares empresariais.\n\nProvedores globais instalam datacenters no Brasil. Multi-cloud oferece flexibilidade e resiliência.",
            "category": "cloud"
        }
    ]
    return texts

# Carregar documentos
documents = create_sample_portuguese_texts()

print(f"Total de documentos: {len(documents)}")

## 3. Processamento e Chunking de Documentos

**Chunking** é o processo de dividir documentos grandes em pedaços menores. Vamos dividir cada documento por **parágrafos**.

### Por que fazer chunking?

- **Precisão**: Chunks menores melhoram a precisão da busca semântica
- **Performance**: Vetores menores são mais eficientes para comparação
- **Contexto**: Cada parágrafo mantém contexto semântico coerente

### Estratégia:
Vamos dividir cada documento pelos caracteres `\n\n` (quebra de linha dupla), que separam parágrafos.

In [ ]:
def chunk_documents(documents: List[Dict]) -> List[Dict]:
    """
    Divide documentos em chunks (parágrafos).
    
    Args:
        documents: Lista de documentos originais
        
    Returns:
        Lista de chunks com metadados preservados
    """
    chunks = []
    
    for doc in documents:
        # Divide o conteúdo por parágrafos (quebra dupla de linha)
        paragraphs = [p.strip() for p in doc['content'].split('\n\n') if p.strip()]
        
        # Cria um chunk para cada parágrafo
        for idx, paragraph in enumerate(paragraphs):
            chunk = {
                'chunk_id': f"{doc['id']}_p{idx+1}",  # ID único do chunk
                'doc_id': doc['id'],                   # ID do documento original
                'title': doc['title'],
                'category': doc['category'],
                'paragraph_num': idx + 1,
                'total_paragraphs': len(paragraphs),
                'content': paragraph,
                'char_count': len(paragraph)
            }
            chunks.append(chunk)
    
    return chunks

# Processar documentos
chunks = chunk_documents(documents)

In [ ]:
# Processar documentos
chunks = chunk_documents(documents)

print(f"Chunking concluído.")
print(f"Estatísticas:")
print(f"\t- Documentos originais: {len(documents)}")
print(f"\t- Total de chunks: {len(chunks)}")
print(f"\t- Média de chunks por documento: {len(chunks)/len(documents):.1f}")

# Mostrar exemplo de documento ANTES do chunking
print("\n\nANTES DO CHUNKING - Documento Original:\n")
print(f"ID: {documents[0]['id']}")
print(f"Título: {documents[0]['title']}")
print(f"Conteúdo completo:\n{documents[0]['content']}")

# Mostrar os chunks gerados deste documento
print("\n\nDEPOIS DO CHUNKING - Chunks Gerados:")
doc_chunks = [c for c in chunks if c['doc_id'] == '001']
for chunk in doc_chunks:
    print(f"\n\t- Chunk ID: {chunk['chunk_id']}")
    print(f"\t- Parágrafo: {chunk['paragraph_num']}/{chunk['total_paragraphs']}")
    print(f"\t- Tamanho: {chunk['char_count']} caracteres")
    print(f"\t- Conteúdo: {chunk['content'][:100]}...")
    print("-" * 100)

## 4. Geração de Embeddings

Vamos transformar texto em **vetores numéricos** (embeddings) usando o modelo **Sentence-BERT**.

### O que são Embeddings?

Embeddings são representações numéricas de texto que capturam o **significado semântico**:
- Textos similares têm vetores próximos no espaço vetorial
- Cada dimensão captura diferentes aspectos do significado
- Permitem comparação matemática entre textos

### Modelo Utilizado:
[**`paraphrase-multilingual-MiniLM-L12-v2`**](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2)
- Otimizado para português e outras línguas
- Gera vetores de **384 dimensões**
- Treinado em milhões de pares de sentenças

In [ ]:
# Carregar modelo de embeddings
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# Informações sobre o modelo
print(f"Informações do Modelo:")
print(f"\t- Dimensões do vetor: {embedding_model.get_sentence_embedding_dimension()}")

### Demonstração: Texto para Vetor

Vamos ver **passo a passo** como um chunk de texto é transformado em vetor.

In [ ]:
# Selecionar um chunk de exemplo
example_chunk = chunks[0]

In [ ]:
print("ANTES: TEXTO ORIGINAL\n")
print(f"\t- Chunk ID: {example_chunk['chunk_id']}")
print(f"\t- Categoria: {example_chunk['category']}")
print(f"\t- Tipo de dado: {type(example_chunk['content'])}")
print(f"\nConteúdo:")
print(f'"{example_chunk["content"]}"')

In [ ]:
# Gerar embedding
embedding = embedding_model.encode(example_chunk['content'])

In [ ]:
print("DEPOIS: VETOR DE EMBEDDING")
print(f"\t- Tipo de dado: {type(embedding)}")
print(f"\t- Dimensões: {embedding.shape}")
print(f"\nPrimeiros 10 valores do vetor:")
print(embedding[:10])
print(f"\nÚltimos 10 valores do vetor:")
print(embedding[-10:])

In [ ]:
# Visualizar distribuição dos valores
plt.figure(figsize=(12, 4))
plt.hist(embedding, bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribuição dos Valores no Vetor de Embedding', fontsize=14, fontweight='bold')
plt.xlabel('Valor')
plt.ylabel('Frequência')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Interpretação:
- Cada número representa uma `feature` semântica abstraída do texto
- Textos similares terão vetores com valores próximos
- A distância entre vetores mede similaridade semântica

### Gerando Embeddings para Todos os Chunks

Agora vamos processar todos os chunks e gerar seus embeddings.

In [ ]:
# Gerar embeddings para todos os chunks
print("Gerando embeddings para todos os chunks...")
texts_to_encode = [chunk['content'] for chunk in chunks]
all_embeddings = embedding_model.encode(texts_to_encode, show_progress_bar=True)

In [ ]:
print(f"Shape da matriz de embeddings: {all_embeddings.shape}")
print(f"\t- {all_embeddings.shape[0]} chunks")
print(f"\t- {all_embeddings.shape[1]} dimensões por vetor")

In [ ]:
# Adicionar embeddings aos chunks
for chunk, embedding in zip(chunks, all_embeddings):
    chunk['embedding'] = embedding.tolist()

## 5. Inicializar ChromaDB e Criar Coleção

Vamos configurar o **ChromaDB** para armazenar nossos embeddings.

### O que é ChromaDB?

ChromaDB é um banco de dados vetorial que:
- Armazena embeddings eficientemente
- Realiza buscas de similaridade rápidas
- Mantém metadados associados aos vetores

In [ ]:
# Criar cliente ChromaDB (em memória para demonstração)
chroma_client = chromadb.Client()

# Criar coleção
collection_name = "documentos_portugues"

# Deletar coleção se já existir (para re-executar o notebook)
try:
    chroma_client.delete_collection(collection_name)
except:
    pass

collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"description": "Documentos em português", "hnsw:space": "cosine"}
)

print(f"ChromaDB inicializado.")
print(f"Coleção criada: '{collection_name}'")
print(f"\nConfigurações da coleção:")
print(f"\t- Nome: {collection.name}")
print(f"\t- Metadados: {collection.metadata}")
print(f"\t- Métrica de distância: Cosine")
print(f"\t- Quantidade de documentos: {collection.count()}")

### Inserindo Chunks no ChromaDB

Vamos adicionar todos os chunks com seus embeddings e metadados.

In [ ]:
# Preparar dados para inserção
ids = [chunk['chunk_id'] for chunk in chunks]
embeddings_list = [chunk['embedding'] for chunk in chunks]
documents_list = [chunk['content'] for chunk in chunks]
metadatas = [
    {
        'doc_id': chunk['doc_id'],
        'title': chunk['title'],
        'category': chunk['category'],
        'paragraph_num': chunk['paragraph_num'],
        'char_count': chunk['char_count']
    }
    for chunk in chunks
]

print("Inserindo chunks no ChromaDB...")
collection.add(
    ids=ids,
    embeddings=embeddings_list,
    documents=documents_list,
    metadatas=metadatas
)

print(f"Chunks inseridos com sucesso.")
print(f"\nEstatísticas da coleção:")
print(f"\t- Total de documentos: {collection.count()}")
print(f"\t- IDs de exemplo: {ids[:3]}")
print(f"\nO que foi armazenado:")
print(f"\t- Embeddings vetoriais ({all_embeddings.shape[1]} dimensões)")
print(f"\t- Textos originais dos chunks")
print(f"\t- Metadados (categoria, título, etc)")
print(f"\t- IDs únicos para cada chunk")

## 6. Busca Semântica

Agora vamos realizar uma **busca semântica** e entender cada etapa do processo!

### Como funciona a busca semântica?

1. **Query:** Convertida em embedding
2. **Comparação:** Calcula similaridade com todos os vetores armazenados
3. **Ranking:** Ordena resultados por similaridade
4. **Retorno:** Devolve os k documentos mais similares

In [ ]:
# Definir query de busca
query_text = "inteligência artificial e machine learning em hospitais"

print("ETAPA 1: QUERY ORIGINAL")
print(f'Query: "{query_text}"')
print(f"Tipo: {type(query_text)}")
print(f"Tamanho: {len(query_text)} caracteres")

In [ ]:
# Converter query em embedding
query_embedding = embedding_model.encode(query_text)

print("ETAPA 2: QUERY COMO VETOR")
print(f"Dimensões: {query_embedding.shape}")
print(f"Primeiros 10 valores: {query_embedding[:10]}")
print(f"\nAgora a query está no mesmo 'espaço vetorial' que os documentos.")

In [ ]:
# Realizar busca no ChromaDB
results = collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=3  # Top 3 resultados mais similares
)

print("ETAPA 3: RESULTADOS DA BUSCA")
print(f"Encontrados: {len(results['ids'][0])} resultados")

In [ ]:
# Mostrar resultados detalhados
for idx, (doc_id, document, metadata, distance) in enumerate(zip(
    results['ids'][0],
    results['documents'][0],
    results['metadatas'][0],
    results['distances'][0]
), 1):
    similarity_score = (1 - (distance / 2)) * 100  # Converter para porcentagem 0-100%
    print(f"\nResultado #{idx}")
    print(f"\t- Chunk ID: {doc_id}")
    print(f"\t- Título: {metadata['title']}")
    print(f"\t- Categoria: {metadata['category']}")
    print(f"\t- Distância Cosine: {distance:.4f} (menor = melhor)")
    print(f"\t- Match Score: {similarity_score:.1f}%")
    print(f"\t- Conteúdo: {document[:100]}...")
    print("-" * 100)

### Visualizando Similaridade entre Query e Resultados

Vamos calcular e visualizar as similaridades usando **cosseno**.

In [ ]:
# Pegar embeddings dos resultados
result_chunk_ids = results['ids'][0]
result_embeddings = []

for chunk_id in result_chunk_ids:
    chunk = next(c for c in chunks if c['chunk_id'] == chunk_id)
    result_embeddings.append(chunk['embedding'])

result_embeddings = np.array(result_embeddings)

# Calcular similaridade de cosseno
query_emb_2d = query_embedding.reshape(1, -1)
similarities = cosine_similarity(query_emb_2d, result_embeddings)[0]

# Criar visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras de similaridade
categories = [results['metadatas'][0][i]['category'] for i in range(len(similarities))]
colors = plt.cm.viridis(np.linspace(0, 1, len(similarities)))

bars = ax1.barh(range(len(similarities)), similarities, color=colors)
ax1.set_yticks(range(len(similarities)))
ax1.set_yticklabels([f"Resultado {i+1}\n({cat})" for i, cat in enumerate(categories)])
ax1.set_xlabel('Similaridade de Cosseno', fontweight='bold')
ax1.set_title('Similaridade: Query vs Resultados', fontweight='bold')
ax1.set_xlim(0, 1)

for i, (bar, sim) in enumerate(zip(bars, similarities)):
    ax1.text(sim + 0.01, i, f'{sim:.4f}', va='center')

# Heatmap de comparação
comparison_matrix = np.zeros((1, len(similarities)))
comparison_matrix[0] = similarities

im = ax2.imshow(comparison_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
ax2.set_yticks([0])
ax2.set_yticklabels(['Query'])
ax2.set_xticks(range(len(similarities)))
ax2.set_xticklabels([f'R{i+1}' for i in range(len(similarities))])
ax2.set_title('Mapa de Calor de Similaridade', fontweight='bold')

for i in range(len(similarities)):
    ax2.text(i, 0, f'{similarities[i]:.3f}', ha='center', va='center', fontweight='bold')

plt.colorbar(im, ax=ax2, label='Similaridade')
plt.tight_layout()
plt.show()

print("Interpretação:")
print("\t- Valores próximos de 1.0 = Alta similaridade")
print("\t- Valores próximos de 0.0 = Baixa similaridade")
print(f"\t- Resultado mais similar: {similarities.max():.4f}")

## 7. Experimentos Adicionais - Diferentes Queries

Vamos testar a busca semântica com diferentes queries e ver como o sistema responde!

In [ ]:
# Diferentes queries para testar
test_queries = [
    "pagamentos digitais e bancos",
    "energia solar e sustentabilidade",
    "ensino online e educação",
    "segurança de dados e proteção"
]

print("TESTANDO MÚLTIPLAS QUERIES:")

results_summary = []

for query in test_queries:
    # Gerar embedding da query
    query_emb = embedding_model.encode(query)
    
    # Buscar no ChromaDB
    search_results = collection.query(
        query_embeddings=[query_emb.tolist()],
        n_results=1  # Top 1 resultado
    )
    
    # Armazenar resultados
    distance_val = search_results['distances'][0][0]
    similarity_score = (1 - (distance_val / 2)) * 100
    top_result = {
        'query': query,
        'doc_id': search_results['ids'][0][0],
        'title': search_results['metadatas'][0][0]['title'],
        'category': search_results['metadatas'][0][0]['category'],
        'distance': distance_val,
        'similarity_score': similarity_score,
        'content': search_results['documents'][0][0][:100]
    }
    results_summary.append(top_result)
    
    print(f"\nQuery: \"{query}\"")
    print(f"\t- Melhor match: {top_result['title']}")
    print(f"\t- Categoria: {top_result['category']}")
    print(f"\t- Match Score: {similarity_score:.1f}%")
    print(f"\t- Trecho: {top_result['content']}...")

In [ ]:
# Criar visualização comparativa
fig, ax = plt.subplots(figsize=(12, 6))

queries_short = [q[:30] + '...' if len(q) > 30 else q for q in test_queries]
similarity_scores = [r['similarity_score'] for r in results_summary]
categories = [r['category'] for r in results_summary]

bars = ax.barh(range(len(queries_short)), similarity_scores)

# Colorir por categoria
category_colors = {cat: plt.cm.Set3(i) for i, cat in enumerate(set(categories))}
for bar, cat in zip(bars, categories):
    bar.set_color(category_colors[cat])

ax.set_yticks(range(len(queries_short)))
ax.set_yticklabels(queries_short)
ax.set_xlabel('Match Score (%)', fontweight='bold')
ax.set_title('Match Score das Queries com Melhor Resultado', fontweight='bold')
ax.set_xlim(0, 100)

for i, (bar, score, cat) in enumerate(zip(bars, similarity_scores, categories)):
    ax.text(score + 2, i, f'{score:.1f}% ({cat})', va='center', fontsize=10)

plt.tight_layout()
plt.show()

## 8. Análise de Metadados - Filtragem por Categoria

ChromaDB permite filtrar resultados por metadados. Vamos buscar documentos de uma categoria específica!

In [ ]:
# Busca com filtro de categoria
query_filtered = "tecnologia e inovação"
target_category = "saude"

print(f"Query: \"{query_filtered}\"")
print(f"Filtrando apenas categoria: {target_category}\n")

In [ ]:
# Busca SEM filtro
results_no_filter = collection.query(
    query_embeddings=[embedding_model.encode(query_filtered).tolist()],
    n_results=3
)

# Busca COM filtro
results_with_filter = collection.query(
    query_embeddings=[embedding_model.encode(query_filtered).tolist()],
    n_results=3,
    where={"category": target_category}  # Filtro por metadado
)

In [ ]:
print("RESULTADOS SEM FILTRO (Top 3 geral)")
for i in range(len(results_no_filter['ids'][0])):
    distance = results_no_filter['distances'][0][i]
    similarity_score = (1 - (distance / 2)) * 100
    print(f"{i+1}. {results_no_filter['metadatas'][0][i]['title']}")
    print(f"\t- Categoria: {results_no_filter['metadatas'][0][i]['category']}")
    print(f"\t- Match Score: {similarity_score:.1f}%\n")

print(f"\nRESULTADOS COM FILTRO (Top 3 em '{target_category}')")
for i in range(len(results_with_filter['ids'][0])):
    distance = results_with_filter['distances'][0][i]
    similarity_score = (1 - (distance / 2)) * 100
    print(f"{i+1}. {results_with_filter['metadatas'][0][i]['title']}")
    print(f"\t- Categoria: {results_with_filter['metadatas'][0][i]['category']}")
    print(f"\t- Match Score: {similarity_score:.1f}%\n")

print("Vantagem: Combina busca semântica com filtros estruturados.")